In [1]:
import numpy as np
import pandas as pd
from abc import ABC, abstractmethod
from typing import Generator, Tuple
import scipy.stats as ss
from scipy.stats import norm
import h2o
from h2o.automl import H2OAutoML
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, KFold, TimeSeriesSplit
from joblib import Parallel, delayed
import math
import itertools as itt
import numpy as np
import itertools as itt
from sklearn.linear_model import LogisticRegression

from h2o.automl import H2OAutoML
h2o.init(verbose=False)
h2o.no_progress()

def calc_log_returns(close: pd.Series, positions: int = 1) -> pd.Series:
    return (np.log(close).diff() * positions.shift()).fillna(0)

def calc_sharpe(pnl):
    pnl.dropna(inplace = True) 
    annualised_pnl = pnl.resample('D').sum() 
    sharpe = float(annualised_pnl.mean() / annualised_pnl.std() * np.sqrt(252))
    return sharpe

def print_ranges(indices):
    if len(indices) == 0:
        return
    
    start = indices[0] + 1
    for i in range(1, len(indices)):
        if indices[i] != indices[i - 1] + 1:
            print(f'   > {start}-{indices[i - 1] + 1}')
            start = indices[i] + 1
    print(f'   > {start}-{indices[-1] + 1}')

close = pd.read_csv('../data/forex_light/EURJPY.csv')
close.set_index('Date', inplace=True)
close.index = pd.to_datetime(close.index)
close = close.resample('1H').last().ffill()
close = close.iloc[:, 0]

features = pd.read_csv('features.csv')
target = pd.read_csv('target.csv')
times = pd.read_csv('times.csv')
features = features.set_index('Date')
target = target.set_index('Date')
features.index = pd.to_datetime(features.index)
target.index = pd.to_datetime(target.index)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, train_size=0.8, shuffle=False)
train = pd.concat([features_train, target_train], axis=1)
test = pd.concat([features_test, target_test], axis=1)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, train_size=0.8, shuffle=False)
train = pd.concat([features_train, target_train], axis=1)
test = pd.concat([features_test, target_test], axis=1)

FileNotFoundError: [Errno 2] No such file or directory: '../data/forex_light/EURJPY.csv'

In [ ]:
train2 = train.iloc[:400]
train2

## Before splitting train/validations

In [ ]:
def identify_best_models():
    train_h2o = h2o.H2OFrame(train2)

    y = 'label'
    x = train2.columns.tolist()
    x.remove(y)
    train_h2o[y] = train_h2o[y].asfactor()
    excluded_algos = ['StackedEnsemble']

    h2o_automl = H2OAutoML(max_runtime_secs=40, max_models=2, exclude_algos=excluded_algos)
    h2o_automl.train(x=x, y=y, training_frame=train_h2o)
    leaderboard = h2o_automl.leaderboard.as_data_frame(use_multi_thread=True)
